In [1]:
pip install eventlet


  Using cached eventlet-0.33.0-py2.py3-none-any.whl (226 kB)
  Using cached dnspython-2.2.1-py3-none-any.whl (269 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install flask-socketio


     |████████████████████████████████| 56 kB 3.7 MB/s eta 0:00:011
     |████████████████████████████████| 52 kB 1.8 MB/s  eta 0:00:01
  Using cached bidict-0.22.0-py3-none-any.whl (36 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import eventlet
import socketio
import cv2
import io
import numpy as np
from PIL import Image
import base64

from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
import cv2


import imutils
import time
import dlib

from imutils.video import VideoStream
from imutils import face_utils

# calculate lip distance
def lip_distance(shape):
    top_lip = shape[50:53]
    top_lip = np.concatenate((top_lip, shape[61:64]))
    low_lip = shape[56:59]
    low_lip = np.concatenate((low_lip, shape[65:68]))
    top_mean = np.mean(top_lip, axis=0)
    low_mean = np.mean(low_lip, axis=0)
    distance = abs(top_mean[1] - low_mean[1])
    return distance

# # E vowel threshold
# E_LT = 9.0
# E_UT = 12.0

HAARCASCADE = "haarcascade_frontalface_default.xml"
MODEL = "model.h5"

# initialize dlib's face detector (HOG-based) and then create the facial landmark predictor
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

sio = socketio.Server(cors_allowed_origins='*')

@sio.event
def my_message(data):
    print('message received with ', data)
    sio.emit('my response', {'response': 'my response'})

@sio.on('image')
def on_message(arg1, data_image):
    sbuf = io.StringIO()
    sbuf.write(data_image)

    # decode and convert into image
    b = io.BytesIO(base64.b64decode(data_image))
    pimg = Image.open(b)

    ## converting RGB to BGR, as opencv standards
    frame = cv2.cvtColor(np.array(pimg), cv2.COLOR_RGB2BGR)
    # emit the frame back

    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	# detect faces in the grayscale frame
    rects = detector(gray, 0)
 	# loop over the face detections
    for rect in rects:
    	# determine the facial landmarks for the face region, then convert the facial landmark (x, y)-coordinates to a NumPy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

		# calculate lip distance 
        distance = lip_distance(shape)

		# draw lip contours
        lip = shape[48:60]
        cv2.drawContours(frame, [lip], -1, (0, 255, 0), 1)

			
		# thresholds and frame counters
        if E_LT < distance < E_UT:
            cv2.putText(frame, "E detected: {:.2f}".format(distance), (300, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # frame = cv2.flip(frame, 1)
    imgencode = cv2.imencode('.jpg', frame)[1]
    # cv2.imshow("Frame", frame)
    # cv2.waitKey(0)
    # print(arg1)

    # base64 encode
    stringData = base64.b64encode(imgencode).decode('utf-8')
    b64_src = 'data:image/jpg;base64,'
    stringData = b64_src + stringData

    sio.emit('response_back', stringData)

# wrap with a WSGI application
app = socketio.WSGIApp(sio)

if __name__ == '__main__':
    eventlet.wsgi.server(eventlet.listen(('localhost', 5000)), app)


[INFO] loading facial landmark predictor...


(1465) wsgi starting up on http://127.0.0.1:5000
(1465) accepted ('127.0.0.1', 51233)
127.0.0.1 - - [16/Apr/2022 12:30:25] "GET /socket.io/?EIO=4&transport=polling&t=O0nB3yD HTTP/1.1" 200 313 0.005367
127.0.0.1 - - [16/Apr/2022 12:30:25] "POST /socket.io/?EIO=4&transport=polling&t=O0nB3zY&sid=416r7iDPVdGanNgsAAAA HTTP/1.1" 200 202 0.014696
(1465) accepted ('127.0.0.1', 51235)
(1465) accepted ('127.0.0.1', 51236)
127.0.0.1 - - [16/Apr/2022 12:30:25] "GET /socket.io/?EIO=4&transport=polling&t=O0nB3zb&sid=416r7iDPVdGanNgsAAAA HTTP/1.1" 200 248 0.001108
127.0.0.1 - - [16/Apr/2022 12:30:25] "GET /socket.io/?EIO=4&transport=polling&t=O0nB3-F&sid=416r7iDPVdGanNgsAAAA HTTP/1.1" 200 216 0.000505
127.0.0.1 - - [16/Apr/2022 12:30:25] "GET /socket.io/?EIO=4&transport=polling&t=O0nB3-h&sid=416r7iDPVdGanNgsAAAA HTTP/1.1" 200 216 0.000291
127.0.0.1 - - [16/Apr/2022 12:32:22] "GET /socket.io/?EIO=4&transport=websocket&sid=416r7iDPVdGanNgsAAAA HTTP/1.1" 200 0 117.463171
(1465) accepted ('127.0.0.1', 51